In [2]:
# autoreload
%load_ext autoreload
%autoreload 2

In [3]:
import torch
from transformers import AutoModel, AutoTokenizer
from copy import deepcopy
from graphs.llama_graph import TransformerEncoderGraph


In [4]:
# this is a model
model_name = "HuggingFaceTB/SmolLM-135M-Instruct"
# not used yet
# tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
model.eval()
state_dict = model.state_dict()

print(model)

# defaults merge the minimum so we need to add flags or change defaults
# example = qk=True, merge_type='all', classifier=??
#graph1 = TransformerEncoderGraph(deepcopy(model)).graphify()

LlamaModel(
  (embed_tokens): Embedding(49152, 576, padding_idx=2)
  (layers): ModuleList(
    (0-29): 30 x LlamaDecoderLayer(
      (self_attn): LlamaSdpaAttention(
        (q_proj): Linear(in_features=576, out_features=576, bias=False)
        (k_proj): Linear(in_features=576, out_features=192, bias=False)
        (v_proj): Linear(in_features=576, out_features=192, bias=False)
        (o_proj): Linear(in_features=576, out_features=576, bias=False)
        (rotary_emb): LlamaRotaryEmbedding()
      )
      (mlp): LlamaMLP(
        (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
        (up_proj): Linear(in_features=576, out_features=1536, bias=False)
        (down_proj): Linear(in_features=1536, out_features=576, bias=False)
        (act_fn): SiLU()
      )
      (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
    )
  )
  (norm): LlamaRMSNorm((576,), eps=1e-05)
  (rotary_emb): LlamaRotaryEmbedding

In [16]:
from torchviz import make_dot


In [18]:
input_ids = torch.randint(0, 100, (1, 10))
input_ids = input_ids.to(model.device)
output = model(input_ids)

In [31]:
def forward_hook(module, input, output):
    return output

activation = {}
model.layers[3].register_forward_hook(lambda m, i, o: activation.update({'layer3': o}))
input_ids = torch.randint(0, 100, (1, 10))
input_ids = input_ids.to(model.device)
output = model(input_ids)

In [35]:
#output.last_hidden_state
dot = make_dot(activation['layer3'][0], params=dict(model.named_parameters()))
dot.render("llama_graph", format="png", cleanup=True)


'llama_graph.png'

In [12]:
params = []
for name,param in model.named_parameters():
    tmp_name = name
    if "layers." in name:
        # find the index of the second '.'
        second_dot_index = name.find('.', name.find('.') + 1)
        tmp_name = name[second_dot_index+1:]
    params.append(tmp_name)

unique_params = set(params)
print(unique_params)



{'mlp.up_proj.weight', 'self_attn.v_proj.weight', 'mlp.gate_proj.weight', 'embed_tokens.weight', 'self_attn.q_proj.weight', 'norm.weight', 'input_layernorm.weight', 'post_attention_layernorm.weight', 'self_attn.o_proj.weight', 'mlp.down_proj.weight', 'self_attn.k_proj.weight'}


In [13]:
for param in unique_params:
    print(param)

mlp.up_proj.weight
self_attn.v_proj.weight
mlp.gate_proj.weight
embed_tokens.weight
self_attn.q_proj.weight
norm.weight
input_layernorm.weight
post_attention_layernorm.weight
self_attn.o_proj.weight
mlp.down_proj.weight
self_attn.k_proj.weight


In [12]:
module_name_set = set()
for name, module in model.named_modules():
    if name == "layers":
        continue
    if "layers" in name and len(name.split(".")) == 2:
        #print("skipping ", name)
        continue
    if "layers" in name:
        second_dot_index = name.find('.', name.find('.') + 1)
        tmp_name = name[second_dot_index+1:]
        if len(tmp_name) > 0:
            module_name_set.add(tmp_name)
    else:
        if name == "":
            continue
        module_name_set.add(name)
print(module_name_set)


{'mlp.act_fn', 'norm', 'self_attn.v_proj', 'mlp.down_proj', 'embed_tokens', 'self_attn.rotary_emb', 'mlp', 'self_attn.o_proj', 'input_layernorm', 'post_attention_layernorm', 'mlp.gate_proj', 'rotary_emb', 'self_attn.q_proj', 'self_attn', 'self_attn.k_proj', 'mlp.up_proj'}


In [14]:
for name in sorted(list(module_name_set)):
    print(name)


embed_tokens
input_layernorm
mlp
mlp.act_fn
mlp.down_proj
mlp.gate_proj
mlp.up_proj
norm
post_attention_layernorm
rotary_emb
self_attn
self_attn.k_proj
self_attn.o_proj
self_attn.q_proj
self_attn.rotary_emb
self_attn.v_proj


In [15]:
graph1 = TransformerEncoderGraph(deepcopy(model), modules={})

In [16]:
graph1 = graph1.graphify()
# todo: figure out when postfix and prefix are used
# todo: implement adding embedding (trivial)
# todo: implement llama decoder block
# todo: repeat llama decoder block for all layers
# todo: implment final 2 layers
# todo: figure out what we do for any output layers
# todo: check how to use positional embeddings if there are any after embedding layer

KeyError: 'emb'

In [38]:
for x in named_modules_dict["layers.29.mlp.up_proj"].parameters():
    print(x.shape)
    


torch.Size([1536, 576])
